In [ ]:
!pip install together

In [3]:
import json
from typing import List, Dict

import requests
from together import Together
from transformers import AutoTokenizer

# Предобработка входных данных

В данном задании мы будем ходить в онлайн модель. Предлагается все также ходить в together.ai, т.к. они дают $5 кредита при регистрации.

Вначале давайте руками поиграемся с API, посмотрим, как походы в API соотносятся с тем, что мы делали в домашнем задании "Доступные LLM"

In [4]:
tokenizer = AutoTokenizer.from_pretrained("NousResearch/Meta-Llama-3.1-70B")

tokenizer_config.json:   0%|          | 0.00/50.5k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.09M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/301 [00:00<?, ?B/s]

In [5]:
tokenizer.bos_token, tokenizer.eos_token

('<|begin_of_text|>', '<|end_of_text|>')

In [6]:
tokenizer.start_header_token = '<|start_header_id|>'
tokenizer.end_header_token = '<|end_header_id|>'
tokenizer.eot_token = '<|eot_id|>'

## Ручное форматирование промпта - 5 баллов

Давайте попробуем собрать вход для llama3.1 руками, для этого допишем функцию `format_messages_to_prompt`.
Она принимает messages - массив словарей, где указаны роли и текст сообщений, а возвращает она текст в формате, который нужно подать модели.

Например для истории сообщений

```python
messages = [
    {"role": "system", "content": "Some system message"},
    {"role": "user", "content": "This is a message from the user"},
    {"role": "assistant", "content": "this is a mesage from the assistant"}
]
```

должен выдаваться итоговый промпт

```text
<|begin_of_text|><|start_header_id|>system<|end_header_id|>

Some system message<|eot_id|><|start_header_id|>user<|end_header_id|>

This is a message from the user<|eot_id|><|start_header_id|>assistant<|end_header_id|>

this is a mesage from the assistant<|eot_id|>
```

Что важно:
1. Текст начинается со спецтокена bos
2. Дальше идет заголовок start_header_id + end_header_id, которые содержат роль
3. Дальше после \n\n идет текст, заканчивающийся на eot_id
4. Дальше следующий заголовок с новой ролью и т.д.

**Важно** - в данной функции нельзя использовать `tokenizer.apply_chat_template`

In [7]:
def format_messages_to_prompt(messages: List[Dict[str, str]]) -> str:
    prompt = f"{tokenizer.bos_token}"
    for message in messages:
        prompt += f"""{tokenizer.start_header_token}{message['role']}{tokenizer.end_header_token}

{message['content']}{tokenizer.eot_token}"""
    return prompt


messages = [
    {"role": "system", "content": "Some system message"},
    {"role": "user", "content": "This is a message from the user"},
    {"role": "assistant", "content": "this is a mesage from the assistant"}
]

In [8]:
reference_text = """<|begin_of_text|><|start_header_id|>system<|end_header_id|>

Some system message<|eot_id|><|start_header_id|>user<|end_header_id|>

This is a message from the user<|eot_id|><|start_header_id|>assistant<|end_header_id|>

this is a mesage from the assistant<|eot_id|>"""


assert format_messages_to_prompt(messages) == reference_text

Мы также помним, что раньше у нас была `tokenizer.apply_chat_template`. Т.к. у нас неофициальный форк llama3.1, то chat_template в токенайзер нам не завезли, поэтому придется добавить его руками

In [9]:
chat_template = """
{{- bos_token }}
{%- if custom_tools is defined %}
    {%- set tools = custom_tools %}
{%- endif %}
{%- if not tools_in_user_message is defined %}
    {%- set tools_in_user_message = true %}
{%- endif %}
{%- if not date_string is defined %}
    {%- set date_string = "26 Jul 2024" %}
{%- endif %}
{%- if not tools is defined %}
    {%- set tools = none %}
{%- endif %}

{#- This block extracts the system message, so we can slot it into the right place. #}
{%- if messages[0]['role'] == 'system' %}
    {%- set system_message = messages[0]['content']|trim %}
    {%- set messages = messages[1:] %}
{%- else %}
    {%- set system_message = "" %}
{%- endif %}

{#- System message + builtin tools #}
{{- "<|start_header_id|>system<|end_header_id|>\n\n" }}
{%- if builtin_tools is defined or tools is not none %}
    {{- "Environment: ipython\n" }}
{%- endif %}
{%- if builtin_tools is defined %}
    {{- "Tools: " + builtin_tools | reject('equalto', 'code_interpreter') | join(", ") + "\n\n"}}
{%- endif %}
{{- "Cutting Knowledge Date: December 2023\n" }}
{{- "Today Date: " + date_string + "\n\n" }}
{%- if tools is not none and not tools_in_user_message %}
    {{- "You have access to the following functions. To call a function, please respond with JSON for a function call." }}
    {{- 'Respond in the format {"name": function name, "parameters": dictionary of argument name and its value}.' }}
    {{- "Do not use variables.\n\n" }}
    {%- for t in tools %}
        {{- t | tojson(indent=4) }}
        {{- "\n\n" }}
    {%- endfor %}
{%- endif %}
{{- system_message }}
{{- "<|eot_id|>" }}

{#- Custom tools are passed in a user message with some extra guidance #}
{%- if tools_in_user_message and not tools is none %}
    {#- Extract the first user message so we can plug it in here #}
    {%- if messages | length != 0 %}
        {%- set first_user_message = messages[0]['content']|trim %}
        {%- set messages = messages[1:] %}
    {%- else %}
        {{- raise_exception("Cannot put tools in the first user message when there's no first user message!") }}
{%- endif %}
    {{- '<|start_header_id|>user<|end_header_id|>\n\n' -}}
    {{- "Given the following functions, please respond with a JSON for a function call " }}
    {{- "with its proper arguments that best answers the given prompt.\n\n" }}
    {{- 'Respond in the format {"name": function name, "parameters": dictionary of argument name and its value}.' }}
    {{- "Do not use variables.\n\n" }}
    {%- for t in tools %}
        {{- t | tojson(indent=4) }}
        {{- "\n\n" }}
    {%- endfor %}
    {{- first_user_message + "<|eot_id|>"}}
{%- endif %}

{%- for message in messages %}
    {%- if not (message.role == 'ipython' or message.role == 'tool' or 'tool_calls' in message) %}
        {{- '<|start_header_id|>' + message['role'] + '<|end_header_id|>\n\n'+ message['content'] | trim + '<|eot_id|>' }}
    {%- elif 'tool_calls' in message %}
        {%- if not message.tool_calls|length == 1 %}
            {{- raise_exception("This model only supports single tool-calls at once!") }}
        {%- endif %}
        {%- set tool_call = message.tool_calls[0].function %}
        {%- if builtin_tools is defined and tool_call.name in builtin_tools %}
            {{- '<|start_header_id|>assistant<|end_header_id|>\n\n' -}}
            {{- "<|python_tag|>" + tool_call.name + ".call(" }}
            {%- for arg_name, arg_val in tool_call.arguments | items %}
                {{- arg_name + '="' + arg_val + '"' }}
                {%- if not loop.last %}
                    {{- ", " }}
                {%- endif %}
                {%- endfor %}
            {{- ")" }}
        {%- else  %}
            {{- '<|start_header_id|>assistant<|end_header_id|>\n\n' -}}
            {{- '{"name": "' + tool_call.name + '", ' }}
            {{- '"parameters": ' }}
            {{- tool_call.arguments | tojson }}
            {{- "}" }}
        {%- endif %}
        {%- if builtin_tools is defined %}
            {#- This means we're in ipython mode #}
            {{- "<|eom_id|>" }}
        {%- else %}
            {{- "<|eot_id|>" }}
        {%- endif %}
    {%- elif message.role == "tool" or message.role == "ipython" %}
        {{- "<|start_header_id|>ipython<|end_header_id|>\n\n" }}
        {%- if message.content is mapping or message.content is iterable %}
            {{- message.content | tojson }}
        {%- else %}
            {{- message.content }}
        {%- endif %}
        {{- "<|eot_id|>" }}
    {%- endif %}
{%- endfor %}
{%- if add_generation_prompt %}
    {{- '<|start_header_id|>assistant<|end_header_id|>\n\n' }}
{%- endif %}
""".strip()
tokenizer.chat_template = chat_template

## Автоматическая сборка промпта - 5 баллов

Давайте вспомним теперь на деле, как используется chat_template! Попробуем использовать функцию `tokenizer.apply_chat_template`

In [10]:
messages = [
    {"role": "system", "content": "Some system message"},
    {"role": "user", "content": "This is a message from the user"},
    {"role": "assistant", "content": "this is a message from the assistant"}
]

prompt = tokenizer.apply_chat_template(chat_template=tokenizer.chat_template, conversation=messages, tokenize=False) # Ваш код здесь

In [11]:
print(prompt)

<|begin_of_text|><|start_header_id|>system<|end_header_id|>

Cutting Knowledge Date: December 2023
Today Date: 26 Jul 2024

Some system message<|eot_id|><|start_header_id|>user<|end_header_id|>

This is a message from the user<|eot_id|><|start_header_id|>assistant<|end_header_id|>

this is a message from the assistant<|eot_id|>


In [12]:
print(tokenizer.apply_chat_template(chat_template=tokenizer.chat_template, conversation=messages, tokenize=False, add_generation_prompt=True))

<|begin_of_text|><|start_header_id|>system<|end_header_id|>

Cutting Knowledge Date: December 2023
Today Date: 26 Jul 2024

Some system message<|eot_id|><|start_header_id|>user<|end_header_id|>

This is a message from the user<|eot_id|><|start_header_id|>assistant<|end_header_id|>

this is a message from the assistant<|eot_id|><|start_header_id|>assistant<|end_header_id|>




In [13]:
reference_prompt = """<|begin_of_text|><|start_header_id|>system<|end_header_id|>

Cutting Knowledge Date: December 2023
Today Date: 26 Jul 2024

Some system message<|eot_id|><|start_header_id|>user<|end_header_id|>

This is a message from the user<|eot_id|><|start_header_id|>assistant<|end_header_id|>

this is a message from the assistant<|eot_id|>"""

assert prompt == reference_prompt

Обратите внимание, что в заданном chat_template указаны Cutting Knowledge Date, т.е. до данные до какого периода видела модели, и Today Date - захардкоженная дата текущего диалога.

**Вопрос, обязательно напишите свой ответ здесь!**
На что влияет аргумент `add_generation_prompt` в функции `tokenizer.apply_chat_template`? Зачем его использовать?

**Ответ**:
В конце форматированного промпта будет добавлен текст начала ответа ассистента (модели). Это увеличивает шансы того, что модель будет генерировать корректный ответ на запрос вместо, например, продолжения введенного пользователем текста 

## Походы в API - 10 баллов

Теперь давайте посмотрим, как можно ходить в API. Для примера мы будем ходить в together.ai, который щедро предоставляет $5 всем зарегистрировавшимся. Вообще говоря различных провайдеров много, API у них у всех очень похожий, т.к. все мимикрируют под OpenAI.

In [14]:
# Вставьте свой ключ из https://api.together.ai/
API_KEY = "8be9b43c22a28abd4152c6957710d54cec55fc938a0db0fc236fa7b4ead23f59"

Есть несколько способов сходить в API. Можно ходить напрямую через библиотеку **requests**. Допишите post запрос в `url` с данными `data` и заголовками `headers`.

In [15]:
headers = {
    'Authorization': 'Bearer ' + API_KEY,
    'Content-Type': 'application/json',
}
url = "https://api.together.xyz/v1/chat/completions"
messages = [
    {"role": "system", "content": "You are a helpful assistant"},
    {"role": "user", "content": "What is the capital of Britain?"}
]

data = {
    "model": "meta-llama/Meta-Llama-3.1-8B-Instruct-Turbo",
    "messages": messages
}

response = requests.post(url=url, headers=headers, json=data) # Ваш код здесь
model_answer = response.json()['choices'][0]['message']['content'] # Ваш код здесь
assert "london" in model_answer.lower()

In [16]:
print(model_answer)

The capital of the United Kingdom (Britain) is London.


Мы подали messages, дальше они каким-то образом собрались в promt и подались модели. Мы не знаем, какой промпт используется на стороне провайдера. Вспомним про Today Date из предыдущего пункта задания - использует ли его together? Обновляют ли они его сегодняшним днем или оставляют 26 июля? Если обновляют, то по какому часовому поясу?

Чтобы ответы на эти и многие другие вопросы не мучали нас по ночам, можно использовать prompt формат, а именно подать модели текст напрямую на генерацию. Давайте для этого используем `tokenizer.apply_chat_template`. Модель будет принимать текст ровно так, как вы его подадите, без каких-либо предобработок. Подумайте, нужно ли вам использовать аргумент `add_generation_prompt`?

Чтобы послать запрос напрямую, нужно в предыдущем запросе убрать messages, который представляет из себя список словарей, и послать поле prompt - строку с промптом для модели.

In [17]:
headers = {
    'Authorization': 'Bearer ' + API_KEY,
    'Content-Type': 'application/json',
}
url = "https://api.together.xyz/v1/chat/completions"
messages = [
    {"role": "system", "content": "You are a helpful assistant"},
    {"role": "user", "content": "What is the capital of Britain?"}
]

prompt = tokenizer.apply_chat_template(chat_template=tokenizer.chat_template, conversation=messages, tokenize=False, add_generation_prompt=True)

data = {
    "model": "meta-llama/Meta-Llama-3.1-8B-Instruct-Turbo",
    "prompt": prompt # Ваш код здесь
}

response = requests.post(url=url, headers=headers, json=data) # Ваш код здесь
model_answer = response.json()['choices'][0]['message']['content'] # Ваш код здесь
assert "london" in model_answer.lower() and "assistant" not in model_answer.lower()

In [18]:
print(model_answer)

The capital of Britain, also known as the United Kingdom (UK), is London.


## Клиент - 5 баллов

Теперь мы понимаем общую схему взаимодействия с провайдером - они предоставляют апи, куда можно посылать или промтп или историю диалога. При посылке промпта вся ответственность за формат ложится на нас, при посылке messages форматтинг происходит на стороне провайдера, но мы не всегда представляем, как он работает. Выбор в пользу того или иного варианта всегда остается на вас.

Мы использовали выше библиотеку requests, чтобы послать HTTP-запрос на сервера together, однако есть способ и проще - python client. Давайте познакомимся с ним поближе. Для этого давайте используем функцию `client.chat.completions.create`. Также давайте добавим опции сэмплинга, которые в этой функции поддержаны. Их можно посылать и в запросах через requests, но мы здесь и далее будем пользоваться клиентом.
* top_k = 100
* temperature = 0.5
* top_p = 0.9
* repetition_penalty = 1.05

In [19]:
client = Together(api_key=API_KEY)

In [20]:
model_name = "meta-llama/Meta-Llama-3.1-8B-Instruct-Turbo"
messages = [
    {"role": "system", "content": "You are a helpful assistant"},
    {"role": "user", "content": "What is the capital of Britain?"}
]

In [21]:
response = client.chat.completions.create(
    model=model_name,
    messages=messages,
    top_k=100,
    top_p=0.9,
    temperature=0.5,
    repetition_penalty=1.05
)

response_text = response.choices[0].message.content # Ваш код здесь
assert "london" in response_text.lower()

Аналогично посылать просто prompt можно через `client.completions.create`.

## Tools - 5 баллов

Давайте теперь посмотрим, как можно использовать tools в связке с моделями. У нас есть функция, которая входит в базу данных и получает информацию о юзере. Базы данных, конечно же, у нас никакой нет, но у нас есть некоторая функция, которая эмулирует это поведение, так что давайте попробуем ее описать.


In [22]:
def get_user_info_from_db(person_name: str) -> Dict[str, str]:
    database = {
        "ilya": {
            "job": "Software Developer",
            "pets": "dog",
        },
        "farruh": {
            "job": "Senior Data & Solution Architect",
            "hobby": "travelling, hiking",
        },
        "timur": {
            "job": "DeepSchool Founder",
            "city": "Novosibirsk",
        }
    }
    no_info = {"err": f"No info about {person_name}"}
    return database.get(person_name.lower(), no_info)

print(get_user_info_from_db("Timur"))

{'job': 'DeepSchool Founder', 'city': 'Novosibirsk'}


Давайте попробуем описать эту функцию в формате json, чтобы модель могла ее увидеть!
Заполните поля в определении дальше

In [23]:
get_user_info_from_db_tool = {
    "type": "function",
    "function": {
        "name": "get_user_info_from_db",
        "description": "Function to get dictionary with user info from database", # Напишите, что функция делает своими словами
        "parameters": {
            "type": "object",
            "properties": {
                "person_name": {
                    "type": "string",
                    "description": "First name of a person" # Опишите смысл аргумента
                }
            },
            "required": ["person_name"] # укажите обязательные аргументы для функции
        }
    }
}

Теперь давайте подадим это описание в `tokenizer.apply_chat_template`. Обратите внимание на его аргумент `tools`! Не забудьте `add_generation_prompt`, если он нужен.

In [24]:
messages = [
    {"role": "system", "content": "You are a helpful assistant"},
    {"role": "user", "content": "What do you know about Ilya?"}
]
prompt = tokenizer.apply_chat_template(conversation=messages, tools=[get_user_info_from_db_tool], tokenize=False, add_generation_prompt=True) # Ваш код здесь
print(prompt)

<|begin_of_text|><|start_header_id|>system<|end_header_id|>

Environment: ipython
Cutting Knowledge Date: December 2023
Today Date: 26 Jul 2024

You are a helpful assistant<|eot_id|><|start_header_id|>user<|end_header_id|>

Given the following functions, please respond with a JSON for a function call with its proper arguments that best answers the given prompt.

Respond in the format {"name": function name, "parameters": dictionary of argument name and its value}.Do not use variables.

{
    "type": "function",
    "function": {
        "name": "get_user_info_from_db",
        "description": "Function to get dictionary with user info from database",
        "parameters": {
            "type": "object",
            "properties": {
                "person_name": {
                    "type": "string",
                    "description": "First name of a person"
                }
            },
            "required": [
                "person_name"
            ]
        }
    }
}

What do

Давайте пошлем наш запрос в модель. На выбор 2 модели, если не будет работать с 8b, то предлагается посылать в 70b.
Для данного запроса для 8b был подобран работающий `seed=9706540181089681000`, который можно подать в функцию.

Давайте воспользуемся `client.completions.create` для генерации ответа от модели.

In [25]:
model_8b = "meta-llama/Meta-Llama-3.1-8B-Instruct-Turbo"
model_70b = "meta-llama/Meta-Llama-3.1-70B-Instruct-Turbo"

In [26]:
response_8b = client.completions.create(model=model_8b, prompt=prompt, seed=9706540181089681000) # Ваш код здесь
response_70b = client.completions.create(model=model_70b, prompt=prompt, seed=9706540181089681000) # Ваш код здесь

print(response_8b.choices[0].text)
print(response_70b.choices[0].text)

{"name": "get_user_info_from_db", "parameters": {"person_name": "Ilya"}}
{"name": "get_user_info_from_db", "parameters": {"person_name": "Ilya"}}


Если все хорошо, то мы получили ответ от модели, который выглядит как некоторый структурированный вывод, который можно использовать для вызова модели. Давайте попробуем написать функцию, которая принимает ответ модели в "сыром виде", выбирает, какую функцию с какими аргументами вызвать.

Здесь нам поможет FUNCTION_REGISTRY и то, что параметры в функцию можно передавать как словарь, например так
```python
def foo(a, b, c):
    print(a, b, c)

obj = {'b':10, 'c':'lee'}

foo(100, **obj)
```

In [27]:
FUNCTION_REGISTRY = {"get_user_info_from_db": get_user_info_from_db}
# На случай, если модель не генерит function call
reference_answer = """{"name": "get_user_info_from_db", "parameters": {"person_name": "Ilya"}}"""

In [28]:
def parse_function_call(model_answer):
    # Ваш код здесь
    # 1. Проверим, является ли это function call.
    dict_answer = json.loads(model_answer)
    function_call = dict_answer['name']
    if function_call not in FUNCTION_REGISTRY:
        return f"Error: {function_call} is not a function call"
    # 2. Вызов нужной функции с указанными аргументами
    return FUNCTION_REGISTRY[function_call](**dict_answer['parameters'])


assert parse_function_call(reference_answer) == get_user_info_from_db("Ilya")

Теперь давайте попробуем объединить все это в историю диалога и сгенерировать моделью финальный ответ.
Для этого в messages, где хранится наша история диалога нужно добавить
1. Вызов function call моделью с ролью ХХХ (это часть задания, напишите сами)
2. Ответ function call с ролью tool

После этого данный промпт нужно послать модели снова, чтобы получить финальный ответ.
Для этого опять используем `tokenizer.apply_chat_template` и `client.completions.create`.

В зависимости от модели может понадобиться убрать tools (на 8b, 70b должна справиться). Для 8b опять же подобран seed=2017684582943914000

In [29]:
messages = [
    {"role": "system", "content": "You are a helpful assistant"},
    {"role": "user", "content": "What do you know about Ilya?"}
]
# Добавляем ответ модели
model_answer = json.dumps({
    "name": "get_user_info_from_db", 
    "parameters": {"person_name": "Ilya"}
})
messages.append(
    {"role": "assistant", "content": f"Calling function: {model_answer}"}
)
# Добавляем ответ tool
tool_answer = parse_function_call(model_answer)
messages.append(
    {"role": "tool", "name": "get_user_info_from_db", "content": json.dumps(tool_answer)}
)

prompt = tokenizer.apply_chat_template(conversation=messages, tokenize=False, add_generation_prompt=True)
print(prompt)

<|begin_of_text|><|start_header_id|>system<|end_header_id|>

Cutting Knowledge Date: December 2023
Today Date: 26 Jul 2024

You are a helpful assistant<|eot_id|><|start_header_id|>user<|end_header_id|>

What do you know about Ilya?<|eot_id|><|start_header_id|>assistant<|end_header_id|>

Calling function: {"name": "get_user_info_from_db", "parameters": {"person_name": "Ilya"}}<|eot_id|><|start_header_id|>ipython<|end_header_id|>

"{\"job\": \"Software Developer\", \"pets\": \"dog\"}"<|eot_id|><|start_header_id|>assistant<|end_header_id|>




In [30]:
response_8b = client.completions.create(model=model_70b, prompt=prompt, seed=2017684582943914000)
print(response_8b.choices[0].text)

Ilya is a software developer and has a dog.


Теперь давайте посмотрим на chat-API, как обрабатываются function calls там?
Используем для этого уже знакомый `client.chat.completions.create`, обратим внимание на аргумент tools внутри него. Здесь рекомендуется использовать 70b модель. На всякий случай работающий seed=14157400267283583000

In [31]:
messages = [
    {"role": "system", "content": "You are a helpful assistant"},
    {"role": "user", "content": "What do you know about Ilya?"}
]
response = client.chat.completions.create(model=model_70b, messages=messages, tools=[get_user_info_from_db_tool], seed=14157400267283583000) # Ваш код здесь

Мы можем видеть, что у нас не работает предыдущий подход с полем `content`, однако должно было появиться поле `tool_calls`, которое содержит в себе информацию о вызове инструмента

In [32]:
response.choices[0].message.tool_calls

[ToolCalls(id='call_1jbpoaa0eo940h3kyw9tahcq', type='function', function=FunctionCall(name='get_user_info_from_db', arguments='{"person_name":"Ilya"}'), index=0)]

# Использование библиотек

Теперь, когда мы руками прошли весь пути обработки function call можно посмотреть уже на готовые инструменты.
Мы много чего сделали руками:
1. Писали описание функции
2. Обрабатывали ответ
3. Вызывали функцию
4. Возвращали все это в модель

Давайте теперь посмотрим, как оно работает в библиотеках!

**NB** - библиотеки развиваются и вполне, возможно, что к концу курса те интерфейсы, которые мы используем в этом домашнем задании будут уже неактуальны, но я уверен, что знаний и принципов, полученных из этих заданий хватит, чтобы адаптироваться к будущим вызовам!

In [ ]:
! pip install langchain==0.2.16 llama-index-core==v0.11.16 langchain-together llama-index-llms-together==0.2.0

# LangChain - 5 баллов

In [34]:
import os
from langchain_together import ChatTogether
from langchain_core.tools import tool

Давайте ознакомимся с langchain-интеграцией together.ai

In [36]:
os.environ["TOGETHER_API_KEY"] = API_KEY

llm = ChatTogether(
    model=model_70b
)

In [37]:
messages = [
    {"role": "system", "content": "You are a helpful assistant"},
    {"role": "user", "content": "What do you know about Ilya?"}
]
response = llm.invoke(messages)
print(response.content)

Ilya can refer to several things, but I'll provide a few possibilities:

1. **Ilya Kovalchuk**: Ilya Valeryevich Kovalchuk is a Russian professional ice hockey player who currently plays for Avtomobilist Yekaterinburg of the Kontinental Hockey League (KHL). He's a highly skilled winger and has played in the NHL for teams like the Atlanta Thrashers, New Jersey Devils, and Los Angeles Kings.

2. **Ilya Repin**: Ilya Yefimovich Repin was a Russian painter and artist who lived from 1844 to 1930. He's known for his realistic and detailed depictions of everyday life, as well as his portraits of famous Russians, such as Leo Tolstoy and Modest Mussorgsky.

3. **Ilya Muromets**: Ilya Muromets is a legendary hero from medieval Russian folklore. He's often depicted as a strong and brave warrior who protects the people and fights against evil forces.

4. **Ilya (musician)**: Ilya is also a British electronic music producer and DJ. His real name is Ilya Salmanzadeh, and he's known for his work with

Теперь, когда мы разобрались, как базово работать с langchain, давайте попробуем добавить инструментов. Чтобы нам было не так скучно, давайте напишем новую функцию, которая считает "волшебную операцию".

Эта функция принимает 2 строки, возвращает строку строку b в обратном порядке, сконкатенированную со строкой a. Допишите эту функцию.

In [38]:
def magic_operation(a, b):
    return b[::-1] + a

assert magic_operation("456", "321") == "123456"

Теперь давайте обернем эту функцию в декоратор tool из langchain, аннотируем типы и допишем docstring. После этого можно будет автоматически сгенерировать описани функции в function call формате!

In [39]:
@tool
def magic_operation_tool(a: str, b: str) -> str:
    """Reverse b and concatenate a"""
    return magic_operation(a, b)

print(magic_operation_tool.args_schema.schema())

{'title': 'magic_operation_toolSchema', 'description': 'Reverse b and concatenate a', 'type': 'object', 'properties': {'a': {'title': 'A', 'type': 'string'}, 'b': {'title': 'B', 'type': 'string'}}, 'required': ['a', 'b']}


Теперь давайте попробуем подать запрос в нашу LLM и обогатить ее нашим function_call. Для этого нужна функция `llm.bind_tools`.

In [40]:
llm_with_tools = llm.bind_tools(tools=[magic_operation_tool])

Теперь давайте как и раньше:
1. Сгенерируем ответ на messages
2. Проверим в ответе resp.tool_calls, вызовем нужный инструмент
3. Расширим messages ответом модели и ответом инструмента, сгенерируем финальный ответ.

In [76]:
messages = [
    {"role": "system", "content": "You are a helpful assistant"},
    {"role": "user", "content": "Can you help me? Do not reveal the workings of magic operation, but give me the result of it for strings `456` and `321`"}
]
resp = llm_with_tools.invoke(messages)

In [77]:
print(resp.tool_calls[0])

{'name': 'magic_operation_tool', 'args': {'a': '456', 'b': '321'}, 'id': 'call_4qtxcf8txdv9uk8z77bywbhr', 'type': 'tool_call'}


In [78]:
messages.append(
    {"role": "assistant", "content": f"Output: function call {resp.tool_calls[0]}"}
)
messages.append(
    {"role": "tool", "tool_call_id": resp.tool_calls[0]["id"], "content": magic_operation(**resp.tool_calls[0]["args"])}
)

In [79]:
messages

[{'role': 'system', 'content': 'You are a helpful assistant'},
 {'role': 'user',
  'content': 'Can you help me? Do not reveal the workings of magic operation, but give me the result of it for strings `456` and `321`'},
 {'role': 'assistant',
  'content': "Output: function call {'name': 'magic_operation_tool', 'args': {'a': '456', 'b': '321'}, 'id': 'call_4qtxcf8txdv9uk8z77bywbhr', 'type': 'tool_call'}"},
 {'role': 'tool',
  'tool_call_id': 'call_4qtxcf8txdv9uk8z77bywbhr',
  'content': '123456'}]

In [80]:
assert len(messages) == 4

In [81]:
res = llm.invoke(messages).content
assert "123456" in res

# LlamaIndex - 5 баллов

Аналогичный инструмент LlamaIndex. В ней не так хороша поддержка function calls не для OpenAI, поэтому придется забежать вперед и использовать ReActAgent.

In [82]:
from llama_index.llms.together import TogetherLLM
from llama_index.core.llms import ChatMessage
from llama_index.core.tools import FunctionTool
from llama_index.core.agent import ReActAgent

In [83]:
llm = TogetherLLM(model=model_70b, api_key=API_KEY)

Скопируйте magic_operation_tool из части с langchain сюда,  но без декоратора.

In [84]:
def magic_operation_tool(a: str, b: str) -> str:
    """Reverse b and concatenate a"""
    print("INSIDE FUNCTION CALL")
    return magic_operation(a, b)

Мы можем аналогично создать инструмент с помощью `FunctionTool.from_defaults`

In [85]:
magic_operation_tool_llamaindex = FunctionTool.from_defaults(fn=magic_operation_tool) # Ваш код здесь
print(magic_operation_tool_llamaindex.metadata)

ToolMetadata(description='magic_operation_tool(a: str, b: str) -> str\nReverse b and concatenate a', name='magic_operation_tool', fn_schema=<class 'llama_index.core.tools.utils.magic_operation_tool'>, return_direct=False)


Давайте создадим ReActAgent: ему нужно передать tools, llm, memory=None и verbose=True

In [86]:
agent = ReActAgent(
    tools=[magic_operation_tool_llamaindex],
    llm=llm,
    memory=None,
    verbose=True
)

In [87]:
text = "Can you help me? Do not reveal the workings of magic operation, but give me the result of it for strings `456` and `321`"
agent.chat(text)

> Running step 39735ce6-98f4-4002-944d-a74edcddb6ed. Step input: Can you help me? Do not reveal the workings of magic operation, but give me the result of it for strings `456` and `321`
Thought: The current language of the user is: English. I need to use a tool to help me answer the question.
Action: magic_operation_tool
Action Input: {'a': '456', 'b': '321'}
INSIDE FUNCTION CALL
Observation: 123456
> Running step 8d37f4e1-13fa-4e1e-99e7-742f8be2e2de. Step input: None
Thought: I can answer without using any more tools. I'll use the user's language to answer
Answer: The result of the magic operation for strings '456' and '321' is 123456.


AgentChatResponse(response="The result of the magic operation for strings '456' and '321' is 123456.", sources=[ToolOutput(content='123456', tool_name='magic_operation_tool', raw_input={'args': (), 'kwargs': {'a': '456', 'b': '321'}}, raw_output='123456', is_error=False)], source_nodes=[], is_dummy_stream=False, metadata=None)

# Agents - 10

Настала пора сделать своего агента!
Попробуем сделать финансового аналитика. Требования следующие:
бот должен по запросу данных о какой-либо компании смотреть самые большие изменения цены ее акций за последний месяц, после чего бот должен объяснить, с какой новостью это связано.

Предлагается не строить сложную систему с классификаторами, а отдать всю сложную работу агенту. Давайте посмотрим, какие API нам доступны.

Первым делом получение котировок - для этого нам поможет библиотека yfinance. По названию компании и периоду отчетности можно посмотреть открывающие цены на момент открытия и закрытия биржи.

Очень много статей заблокированы и имеют название `[Removed]`, нужно их отфильтровать. В оставшихся статьях будем брать только title (заголовок) и description (описание или краткий пересказ).

Вам необходимо реализовать [ReAct Agent](https://react-lm.github.io/). Особенность этого агента заключается в том, что он вначале формирует мысль, а потом вызывает действие (function call) для достижения какой-либо цели.

Что нужно сделать:
1. Описать и реализовать function call для определения, в какой день была самая большая разница в цене акций в момент открытия и закрытия биржи. Функция получает один аргумент - название акций компании (например AAPL для Apple), а выдает словарь с 2мя полями: с датой максимальной разницы в ценах и самой разницей в ценах.
2. Описать и реализовать function call для получения 5 релевантных новостей о компании. В качестве аргумента принимаются название компании и дата. Ваша задача - сходить в newsapi, получить новости и вернуть 5 случайных новостей, которые произошли не позже чем день торгов. Если новостей меньше 5, то верните столько, сколько получится.
3. После этого агент должен вернуть ответ, в котором постарается аргументировать изменения в цене.


Реализовывать агента можно любым удобным способом, в том числе взять готовые имплементации.
1. [LlamaIndex](https://docs.llamaindex.ai/en/stable/examples/agent/react_agent/) - вдобавок можно посмотреть предыдущее задание, где он уже используется.
2. [Langchain/Langgraph](https://langchain-ai.github.io/langgraph/how-tos/create-react-agent/#code)
3. Написать полностью свою реализацию


Не забудьте, что очень важно описать задачу в промпте: нужно сказать, какие цели у агента и что он должен сделать. У функций должны быть говорящие описания, чтобы LLM без лишних проблем поняла, какие есть функции и когда их использовать. По всем вопросам можно обращаться в наш телеграм-чат в канал "Tools & Agents".


In [ ]:
!pip install yfinance

In [89]:
import yfinance as yf

stock = yf.Ticker("AAPL") # посмотрим котировки APPLE
df = stock.history(period="1mo")
df[["Open", "Close"]]

,Open,Close
Date,,
2025-01-10 00:00:00-05:00,240.009995,236.850006
2025-01-13 00:00:00-05:00,233.529999,234.399994
2025-01-14 00:00:00-05:00,234.750000,233.279999
2025-01-15 00:00:00-05:00,234.639999,237.869995
2025-01-16 00:00:00-05:00,237.350006,228.259995
2025-01-17 00:00:00-05:00,232.119995,229.979996
2025-01-21 00:00:00-05:00,224.000000,222.639999
2025-01-22 00:00:00-05:00,219.789993,223.830002
2025-01-23 00:00:00-05:00,224.740005,223.660004


In [90]:
import numpy as np
import pandas as pd

@tool
def get_day_with_biggest_price_change(stock_ticker: str) -> str:
    """Finds a date in '%Y-%m-%d' format, that represents the day with the biggest open and close price change for the given stock ticker"""
    stock = yf.Ticker(stock_ticker)
    df = stock.history(period="1mo")
    change_series = (df.Open - df.Close).abs()
    max_date = change_series.idxmax()
    return str(max_date.date())

In [91]:
@tool
def get_company_news(company_name: str, date_before: str) -> List[Dict[str, str]]:
    """Returns five (or less if impossible) news titles and descriptions about the company, that was published before the given date"""
    news = []

    api_key = "1a9ea1edb0394f93a7b810601d7bcdcb"
    api_template = f"https://newsapi.org/v2/everything?q={company_name}&apiKey={api_key}&to={date_before}"
    articles = requests.get(api_template).json()['articles']
    for i in range(min(5, len(articles))):
        if articles[i]["title"] != "[Removed]":
            news.append({"title": articles[i]["title"], "description": articles[i]["description"]})

    return news   

Для поиска новостей нам поможет https://newsapi.org/
Можно легко получить свой ключ за короткую регистрацию, дается 1000 запросов в день, каждый запрос может включать в себя ключевое слово и промежуток дат. По бесплатному апи ключу дается ровно 1 месяц, что нам подходит.

In [92]:
api_key = "1a9ea1edb0394f93a7b810601d7bcdcb" # ваш API ключ здесь!
api_template = "https://newsapi.org/v2/everything?q={keyword}&apiKey={api_key}&to={date_from}"

articles = requests.get(api_template.format(keyword="Apple", api_key=api_key, date_from="2025-01-25")).json()

for article in articles["articles"]:
    if article["title"] != "[Removed]":
        print(article["title"])
        print(article["description"])
        break

How to Use Apple’s Genmoji to Create New Kinds of Emojis
Apple Intelligence lets you generate custom emojis when the default options aren’t hitting it.


In [93]:
from langchain_together import ChatTogether
from langchain_core.prompts import ChatPromptTemplate

model = ChatTogether(model=model_70b)

prompt = ChatPromptTemplate.from_messages(
    [
        ("system", "You are a helpful financial analyst. Your aims are to find stock price for a compony, find relative news about the compony and build a connection between price changes and the news"),
        ("user", "{input}"),
        ("placeholder", "{agent_scratchpad}"), # без этой строчки не работает, вроде это необходимость
    ]
)

In [94]:
from langchain.agents import AgentExecutor, create_tool_calling_agent

query = "What is the highest price change of Apple company stocks for a recent month before '2025-01-25'?"
tools = [get_day_with_biggest_price_change, get_company_news]
agent = create_tool_calling_agent(model, tools, prompt)
agent_executor = AgentExecutor(agent=agent, tools=tools)

In [95]:
response = agent_executor.invoke({"input": query})
print(response['output'])

The highest price change of Apple company stocks for a recent month before '2025-01-25' was on '2025-01-31'. 
Here are five news titles and descriptions about Apple that were published before '2025-01-25': 

1. How to Use Apple’s Genmoji to Create New Kinds of Emojis
   Apple Intelligence lets you generate custom emojis when the default options aren’t hitting it.

2. Parallels is testing x86 emulation on Apple silicon Macs
   Parallels version 20.2 has added an early preview of x86 emulation support for Apple silicon Macs, with several limitations that mean most won’t want to use it.

3. The 4.8-Star-Rated 2024 15-Inch Apple MacBook Air Is $250 Off on Amazon for MLK Weekend
   The lightning-fast M3-powered Apple laptop is built for the all-new Apple Intelligence and has an incredible 18-hour battery life.

4. Apple is pausing notification summaries for news in the latest iOS 18.3 beta
   Apple has temporarily stopped showing notification summaries for news and entertainment apps in the

## Все сработало!